In [ ]:
import numpy as np
import pandas as pd
import scipy
import pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_roc_curve
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
seed = 101
data = pd.read_excel("input/MDR-cross-regional.xlsx",index_col=0)
data.head(15000)
data.cross_regional.value_counts()
x=data.drop(['cross_regional'], axis=1)
x.head(15000)
y=data.cross_regional
y.head(15000)

In [ ]:
# 划分训练集和测试集
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()  #实例化


In [ ]:
#定义超参数范围
param_grid = {'n_estimators': range(25, 500, 25), 'max_depth': [None, 5, 10]}

#使用网格搜索来找到最佳超参数组合
grid_search = GridSearchCV(rfc, param_grid)
grid_search.fit(xtrain, ytrain)
print("Best parameter values:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)



In [ ]:
# 创建一个新的随机森林分类器，并设置最佳超参数
rfc = RandomForestClassifier(max_depth=5,n_estimators=175)

In [ ]:
#使用交叉验证进行模型评估
scores = cross_val_score(rfc, x, y, cv=10, scoring='accuracy')
print("Mean score:", scores.mean())
print("Standard deviation:", scores.std())


In [ ]:
# 训练模型并预测
rfc.fit(xtrain, ytrain)
test_predict = rfc.predict(xtest)

In [ ]:
import pandas as pd

# 假设因变量列名为 "Cluster"，训练集名称为 xtrain，测试集名称为 xtest
class_counts_train = ytrain.value_counts()
print('Training set: \n', class_counts_train)

class_counts_test = ytest.value_counts()
print('\nTesting set: \n', class_counts_test)

In [ ]:
importances = rfc.feature_importances_
std = np.std([rfc.feature_importances_ for tree in rfc.estimators_], axis=0)
indices = np.argsort(importances)[::-1]#Print the feature ranking,从大到小排序
print("Feature ranking:")
for f in range(min(3000,xtrain.shape[1])):
    print("%2d) %-*s %f" %(f + 1, 30, xtrain.columns[indices[f]], importances[indices[f]]))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from scipy import stats
from sklearn.metrics import f1_score
import statsmodels.stats.proportion as smp
predictions_test = rfc.predict(xtest)
predictions_train = rfc.predict(xtrain)
# 计算训练集和测试集的F1值
train_f1 = f1_score(ytrain, predictions_train, average='weighted')
test_f1 = f1_score(ytest, predictions_test, average='weighted')

print("训练集 F1值：", train_f1)
print("测试集 F1值：", test_f1)

# 计算训练集和测试集的加权F1值置信区间
ci_low_train, ci_up_train = smp.proportion_confint(sum(predictions_train), len(predictions_train), alpha=0.05, method='normal')
ci_low_test, ci_up_test = smp.proportion_confint(sum(predictions_test), len(predictions_test), alpha=0.05, method='normal')

train_se = (round(ci_low_train, 3), round(ci_up_train, 3))
test_se = (round(ci_low_test, 3), round(ci_up_test, 3))

print("训练集 F1值 95% CI：", train_se)
print("测试集 F1值 95% CI：", test_se)
#因变量是二分类
from sklearn.metrics import cohen_kappa_score
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from scipy import stats
predictions_train = rfc.predict(xtrain)
predictions_test = rfc.predict(xtest)
# 计算训练集上的 Kappa 值
train_kappa = cohen_kappa_score(ytrain, predictions_train)

# 计算训练集 Kappa 值的 95% CI
n1 = np.sum(ytrain==1)
n2 = np.sum(ytrain==0)
p0 = np.sum(predictions_train == ytrain) / len(predictions_train)
pp = (np.sum(ytrain==1) * np.sum(predictions_train==1) + np.sum(ytrain==0) * np.sum(predictions_train==0)) / len(predictions_train)**2
pe = (n1/len(predictions_train) * n1/len(predictions_train)) + (n2/len(predictions_train) * n2/len(predictions_train))
kappa_ci_lower = round((p0 - pp)/(1 - pe), 3)
kappa_ci_upper = round((p0 - pp)/(1 - pe) + 1.96 * np.sqrt((p0*(1-p0)+pp*(1-pp)-2*pe*(p0-pp)) / len(predictions_train)), 3)

# 输出结果
print(f'Training Kappa: {train_kappa:.3f}, 95% CI: ({kappa_ci_lower:.3f}, {kappa_ci_upper:.3f})')

# 计算测试集上的 Kappa 值
test_kappa = cohen_kappa_score(ytest, predictions_test)

# 计算测试集 Kappa 值的 95% CI
n1 = np.sum(ytest==1)
n2 = np.sum(ytest==0)
p0 = np.sum(predictions_test == ytest) / len(predictions_test)
pp = (np.sum(ytest==1) * np.sum(predictions_test==1) + np.sum(ytest==0) * np.sum(predictions_test==0)) / len(predictions_test)**2
pe = (n1/len(predictions_test) * n1/len(predictions_test)) + (n2/len(predictions_test) * n2/len(predictions_test))
kappa_ci_lower = round((p0 - pp)/(1 - pe), 3)
kappa_ci_upper = round((p0 - pp)/(1 - pe) + 1.96 * np.sqrt((p0*(1-p0)+pp*(1-pp)-2*pe*(p0-pp)) / len(predictions_test)), 3)

# 输出结果
print(f'Testing Kappa: {test_kappa:.3f}, 95% CI: ({kappa_ci_lower:.3f}, {kappa_ci_upper:.3f})')
from sklearn.metrics import roc_auc_score
from scipy import stats

# 对测试集进行预测并计算ROC AUC
probabilities_test = rfc.predict_proba(xtest)[:, 1]
roc_auc_test = round(roc_auc_score(ytest, probabilities_test), 3)

# 计算测试集ROC AUC置信区间
n_test = len(ytest)
se_test = (roc_auc_test*(1-roc_auc_test)/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(roc_auc_test - z_score_test*se_test, 3), round(roc_auc_test + z_score_test*se_test, 3))

# 输出结果
print(f'ROC AUC for test set: {roc_auc_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

from sklearn.metrics import roc_auc_score
from scipy import stats

# 对训练集进行预测并计算ROC AUC
probabilities_train = rfc.predict_proba(xtrain)[:, 1]
roc_auc_train = round(roc_auc_score(ytrain, probabilities_train), 3)

# 计算训练集ROC AUC置信区间
n_train = len(ytrain)
se_train = (roc_auc_train*(1-roc_auc_train)/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(roc_auc_train - z_score_train*se_train, 3), round(roc_auc_train + z_score_train*se_train, 3))

# 输出结果
print(f'ROC AUC for train set: {roc_auc_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')

from sklearn.metrics import recall_score
from scipy import stats

# 对测试集进行预测
predictions_test = rfc.predict(xtest)

# 计算测试集召回率，并保留3位小数
recall_test = round(recall_score(ytest, predictions_test), 3)

# 计算测试集置信区间
n_test = len(ytest)
se_test = (recall_test*(1-recall_test)/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(recall_test - z_score_test*se_test, 3), round(recall_test + z_score_test*se_test, 3))

# 输出结果
print(f'Recall for test set: {recall_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

from sklearn.metrics import recall_score
from scipy import stats

# 对训练集进行预测
predictions_train = rfc.predict(xtrain)

# 计算召回率，并保留3位小数
recall_train = round(recall_score(ytrain, predictions_train), 3)

# 计算置信区间
n_train = len(ytrain)
se_train = (recall_train*(1-recall_train)/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(recall_train - z_score_train*se_train, 3), round(recall_train + z_score_train*se_train, 3))

# 输出结果
print(f'Recall for train set: {recall_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')
from sklearn.metrics import recall_score
from scipy import stats

# 对测试集进行预测并计算specificity
predictions_test = rfc.predict(xtest)
specificity_test = round(sum((predictions_test == 0) & (ytest == 0)) / sum(ytest == 0), 3)

# 计算测试集specificity置信区间
n_test = len(ytest)
se_test = (specificity_test*(1-specificity_test)/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(specificity_test - z_score_test*se_test, 3), round(specificity_test + z_score_test*se_test, 3))

# 输出测试集结果
print(f'Specificity for test set: {specificity_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

# 对训练集进行预测并计算specificity
predictions_train = rfc.predict(xtrain)
specificity_train = round(sum((predictions_train == 0) & (ytrain == 0)) / sum(ytrain == 0), 3)

# 计算训练集specificity置信区间
n_train = len(ytrain)
se_train = (specificity_train*(1-specificity_train)/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(specificity_train - z_score_train*se_train, 3), round(specificity_train + z_score_train*se_train, 3))

# 输出训练集结果
print(f'Specificity for train set: {specificity_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')
from sklearn.metrics import precision_score
from scipy import stats

from sklearn.metrics import precision_score
from scipy import stats

# 对测试集进行预测并计算ppv
predictions_test = rfc.predict(xtest)
ppv_test = round(precision_score(ytest, predictions_test), 3)

# 计算测试集ppv置信区间
n_test = len(ytest)
se_test = (ppv_test*(1-ppv_test)/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(ppv_test - z_score_test*se_test, 3), round(ppv_test + z_score_test*se_test, 3))

# 输出测试集结果
print(f'Positive Predictive Value (PPV) for test set: {ppv_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

# 对训练集进行预测并计算ppv
predictions_train = rfc.predict(xtrain)
ppv_train = round(precision_score(ytrain, predictions_train), 3)

# 计算训练集ppv置信区间
n_train = len(ytrain)
se_train = (ppv_train*(1-ppv_train)/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(ppv_train - z_score_train*se_train, 3), round(ppv_train + z_score_train*se_train, 3))

# 输出训练集结果
print(f'Positive Predictive Value (PPV) for train set: {ppv_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')

from scipy import stats

# 对测试集进行预测并计算npv
predictions_test = rfc.predict(xtest)
npv_test = round(sum((predictions_test == 0) & (ytest == 0)) / sum(predictions_test == 0), 3)

# 计算测试集npv置信区间
n_test = len(ytest)
se_test = (npv_test*(1-npv_test)/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(npv_test - z_score_test*se_test, 3), round(npv_test + z_score_test*se_test, 3))

# 输出测试集结果
print(f'Negative Predictive Value (NPV) for test set: {npv_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

# 对训练集进行预测并计算npv
predictions_train = rfc.predict(xtrain)
npv_train = round(sum((predictions_train == 0) & (ytrain == 0)) / sum(predictions_train == 0), 3)

# 计算训练集npv置信区间
n_train = len(ytrain)
se_train = (npv_train*(1-npv_train)/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(npv_train - z_score_train*se_train, 3), round(npv_train + z_score_train*se_train, 3))

# 输出训练集结果
print(f'Negative Predictive Value (NPV) for train set: {npv_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')

from scipy import stats

# 对测试集进行预测并计算plr
predictions_test = rfc.predict(xtest)
tp_test = sum((predictions_test == 1) & (ytest == 1))
fp_test = sum((predictions_test == 1) & (ytest == 0))
tn_test = sum((predictions_test == 0) & (ytest == 0))
fn_test = sum((predictions_test == 0) & (ytest == 1))
plr_test = round((tp_test / (tp_test + fp_test)) / (fn_test / (tn_test + fn_test)), 3)

# 计算测试集plr置信区间
n_test = len(ytest)
se_test = ((1+tp_test/fp_test)*(fn_test/tn_test)*((1-tp_test/(tp_test+fp_test))*tp_test/(tp_test+fp_test)+fn_test/(tn_test+fn_test))/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(plr_test - z_score_test*se_test, 3), round(plr_test + z_score_test*se_test, 3))

# 输出测试集结果
print(f'Positive Likelihood Ratio (PLR) for test set: {plr_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

# 对训练集进行预测并计算plr
predictions_train = rfc.predict(xtrain)
tp_train = sum((predictions_train == 1) & (ytrain == 1))
fp_train = sum((predictions_train == 1) & (ytrain == 0))
tn_train = sum((predictions_train == 0) & (ytrain == 0))
fn_train = sum((predictions_train == 0) & (ytrain == 1))
plr_train = round((tp_train / (tp_train + fp_train)) / (fn_train / (tn_train + fn_train)), 3)

# 计算训练集plr置信区间
n_train = len(ytrain)
se_train = ((1+tp_train/fp_train)*(fn_train/tn_train)*((1-tp_train/(tp_train+fp_train))*tp_train/(tp_train+fp_train)+fn_train/(tn_train+fn_train))/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(plr_train - z_score_train*se_train, 3), round(plr_train + z_score_train*se_train, 3))

# 输出训练集结果
print(f'Positive Likelihood Ratio (PLR) for train set: {plr_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')


from scipy import stats

# 对测试集进行预测并计算nlr
predictions_test = rfc.predict(xtest)
tp_test = sum((predictions_test == 1) & (ytest == 1))
fp_test = sum((predictions_test == 1) & (ytest == 0))
tn_test = sum((predictions_test == 0) & (ytest == 0))
fn_test = sum((predictions_test == 0) & (ytest == 1))
nlr_test = round((fn_test / (tn_test + fn_test)) / (tp_test / (tp_test + fp_test)), 3)

# 计算测试集nlr置信区间
n_test = len(ytest)
se_test = ((1+fn_test/tn_test)*(tp_test/fp_test)*((1-tp_test/(tp_test+fp_test))*fp_test/(tp_test+fp_test)+tn_test/(tn_test+fn_test))/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(nlr_test - z_score_test*se_test, 3), round(nlr_test + z_score_test*se_test, 3))

# 输出测试集结果
print(f'Negative Likelihood Ratio (NLR) for test set: {nlr_test:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_test[0]}, {ci_test[1]})')

# 对训练集进行预测并计算nlr
predictions_train = rfc.predict(xtrain)
tp_train = sum((predictions_train == 1) & (ytrain == 1))
fp_train = sum((predictions_train == 1) & (ytrain == 0))
tn_train = sum((predictions_train == 0) & (ytrain == 0))
fn_train = sum((predictions_train == 0) & (ytrain == 1))
nlr_train = round((fn_train / (tn_train + fn_train)) / (tp_train / (tp_train + fp_train)), 3)

# 计算训练集nlr置信区间
n_train = len(ytrain)
se_train = ((1+fn_train/tn_train)*(tp_train/fp_train)*((1-tp_train/(tp_train+fp_train))*fp_train/(tp_train+fp_train)+tn_train/(tn_train+fn_train))/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(nlr_train - z_score_train*se_train, 3), round(nlr_train + z_score_train*se_train, 3))

# 输出训练集结果
print(f'Negative Likelihood Ratio (NLR) for train set: {nlr_train:.3f}, 95% CI ({confidence*100:.0f}%): ({ci_train[0]}, {ci_train[1]})')


from sklearn.metrics import accuracy_score
from scipy import stats

# 计算测试集accuracy及置信区间
predictions_test = rfc.predict(xtest)
accuracy_test = round(accuracy_score(ytest, predictions_test), 3)
n_test = len(ytest)
se_test = (accuracy_test*(1-accuracy_test)/n_test)**0.5
confidence = 0.95
z_score_test = stats.norm.ppf((1+confidence)/2)
ci_test = (round(accuracy_test - z_score_test*se_test, 3), round(accuracy_test + z_score_test*se_test, 3))
print(f'Test set: Accuracy = {accuracy_test:.3f}, 95% CI ({ci_test[0]}, {ci_test[1]})')

# 计算训练集accuracy及置信区间
predictions_train = rfc.predict(xtrain)
accuracy_train = round(accuracy_score(ytrain, predictions_train), 3)
n_train = len(ytrain)
se_train = (accuracy_train*(1-accuracy_train)/n_train)**0.5
z_score_train = stats.norm.ppf((1+confidence)/2)
ci_train = (round(accuracy_train - z_score_train*se_train, 3), round(accuracy_train + z_score_train*se_train, 3))
print(f'Train set: Accuracy = {accuracy_train:.3f}, 95% CI ({ci_train[0]}, {ci_train[1]})')



In [ ]:
predictions_validation = rfc.predict_proba(xtest)[:,1]
fpr, tpr, _ = roc_curve(ytest, predictions_validation)
roc_auc = auc(fpr, tpr)
plt.title('ROC Validation')
plt.grid(False)  # 将格子去掉
plt.plot(fpr, tpr, 'navy', label='AUC = %0.3f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('output/cluster_test.PDF')
plt.show()

In [ ]:
predictions_validation = rfc.predict_proba(xtrain)[:,1]
fpr, tpr, _ = roc_curve(ytrain, predictions_validation)
roc_auc = auc(fpr, tpr)
plt.title('ROC Validation')
plt.grid(False)  # 将格子去掉
plt.plot(fpr, tpr, 'navy', label='AUC = %0.3f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('output/cluster_train.PDF')
plt.show()